# 바른 형태소 분석기 활용

### 바른 설치


바른을 설치합니다.


바른의 리눅스 설치본을 다운로드 받습니다.

In [ ]:
!curl -LJks -H "uname:$(uname -a)" https://bareun.ai/api/get -o bareun-linux.deb
!ls

**바른을** 설치합니다.

In [2]:
!uname -a

Linux dad0a4c130f4 5.10.147+ #1 SMP Sat Dec 10 16:00:40 UTC 2022 x86_64 x86_64 x86_64 GNU/Linux


In [3]:
!dpkg -i bareun-linux.deb

Selecting previously unselected package bareun.
(Reading database ... 128126 files and directories currently installed.)
Preparing to unpack bareun-v2.0.0.linux-x86_64.deb ...
Unpacking bareun (2.0.0) ...
Setting up bareun (2.0.0) ...
System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to connect to bus: Host is down
System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to connect to bus: Host is down
Created symlink /etc/systemd/system/multi-user.target.wants/bareun.service → /etc/systemd/system/bareun.service.


아래는 바른 모델이 GPU를 사용하기 위해서 다운받는 tensorflow 라이브러리입니다.

상단의 메뉴탭의 **"런타임 > 런타임 유형 변경 > 하드웨어 가속기 > GPU 선택 > 저장"**을 설정하시면 모델의 추론 속도가 더 빨라집니다. 단, 무료버전의 경우 coloab의 일일 사용 제한을 초과하지 않아야 합니다.

기본적으로 CPU를 사용해도 작동합니다.

In [4]:
!curl -O https://storage.googleapis.com/tensorflow/libtensorflow/libtensorflow-gpu-linux-x86_64-2.9.1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  392M  100  392M    0     0  36.4M      0  0:00:10  0:00:10 --:--:-- 39.3M


In [5]:
!tar -C /opt/bareun -xzf libtensorflow-gpu-linux-x86_64-2.9.1.tar.gz

# 바른을 시작합니다.

아래 코드는 BAREUN_ROOT와 LD_LIBRARY_PATH를 설정하고, /opt/bareun/bin/bareun을 백그라운드로 실행하는 코드입니다.
* %env BAREUN_ROOT="/opt/bareun": 환경 변수 BAREUN_ROOT를 "/opt/bareun"으로 설정합니다.
* %env LD_LIBRARY_PATH="/opt/bareun/lib" : 환경변수 LD_LIBRARY_PATH를 "/opt/bareun/lib"으로 설정합니다.
* !BAREUN_ROOT="/opt/bareun" LD_LIBRARY_PATH="/opt/bareun/lib" nohup /opt/bareun/bin/bareun& : bareun을 백그라운드로 실행시킵니다.


In [6]:
%env BAREUN_ROOT="/opt/bareun"
%env LD_LIBRARY_PATH="/opt/bareun/lib"
!BAREUN_ROOT="/opt/bareun" LD_LIBRARY_PATH="/opt/bareun/lib" nohup /opt/bareun/bin/bareun&

env: BAREUN_ROOT="/opt/bareun"
env: LD_LIBRARY_PATH="/opt/bareun/lib"
nohup: appending output to 'nohup.out'


In [7]:
!ps -ef | grep bareun

root        1572       1  0 06:15 ?        00:00:00 /opt/bareun/bin/bareun
root        1573     532  0 06:15 ?        00:00:00 /bin/bash -c ps -ef | grep bareun
root        1575    1573  0 06:15 ?        00:00:00 grep bareun


API키 발급

바른 홈페이지(https://bareun.ai/) 에 회원가입해서 API키를 받습니다.

*   API키 복사
*   API키를 등록합니다.


In [ ]:
!BAREUN_ROOT="/opt/bareun" LD_LIBRARY_PATH="/opt/bareun/lib" /opt/bareun/bin/bareun -reg YOUR-API-KEY

PIP 패키지 설치
pip install bareunpy

In [9]:
!pip install -U bareunpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


바른 API를 사용합니다.

In [10]:
import sys
import bareunpy as brn
import google.protobuf.text_format as tf
API_KEY = "YOUR-API-KEY"
t = brn.Tagger(API_KEY, "localhost", 5656)
res = t.tags(["안녕하세요. 정말 좋은 날씨네요."])
m = res.msg()
tf.PrintMessage(m, out=sys.stdout, as_utf8=True)

sentences {
  text {
    content: "안녕하세요. 정말 좋은 날씨네요."
    length: 18
  }
  tokens {
    text {
      content: "안녕하세요."
      length: 6
    }
    morphemes {
      text {
        content: "안녕하"
        length: 3
      }
      tag: VA
      out_of_vocab: IN_BUILTIN_DICT
    }
    morphemes {
      text {
        content: "시"
        begin_offset: 3
        length: 1
      }
      tag: EP
      probability: 0.99986386
    }
    morphemes {
      text {
        content: "어요"
        begin_offset: 3
        length: 2
      }
      tag: EF
      probability: 0.9903047
    }
    morphemes {
      text {
        content: "."
        begin_offset: 5
        length: 1
      }
      tag: SF
      probability: 0.99277085
    }
    lemma: "안녕하"
    tagged: "안녕하/VA+시/EP+어요/EF+./SF"
  }
  tokens {
    text {
      content: "정말"
      begin_offset: 7
      length: 2
    }
    morphemes {
      text {
        content: "정말"
        begin_offset: 7
        length: 2
      }
      tag: MAG
      probabil

In [11]:
print(f'length of tokens in sentences[0] is {len(m.sentences[0].tokens)}')
print(f'length of morphemes of first token in sentences[0] is {len(m.sentences[0].tokens[0].morphemes)}')
print(f'lemma of first token in sentences[0] is {m.sentences[0].tokens[0].lemma}')
print(f'first morph of first token in sentences[0] is {m.sentences[0].tokens[0].morphemes[0]}')
print(f'tag of first morph of first token in sentences[0] is {m.sentences[0].tokens[0].morphemes[0].tag}')

length of tokens in sentences[0] is 4
length of morphemes of first token in sentences[0] is 4
lemma of first token in sentences[0] is 안녕하
first morph of first token in sentences[0] is text {
  content: "\354\225\210\353\205\225\355\225\230"
  length: 3
}
tag: VA
out_of_vocab: IN_BUILTIN_DICT

tag of first morph of first token in sentences[0] is 38


In [12]:
# get json object
jo = res.as_json()
print(jo)

{'sentences': [{'text': {'content': '안녕하세요. 정말 좋은 날씨네요.', 'length': 18, 'beginOffset': 0}, 'tokens': [{'text': {'content': '안녕하세요.', 'length': 6, 'beginOffset': 0}, 'morphemes': [{'text': {'content': '안녕하', 'length': 3, 'beginOffset': 0}, 'tag': 'VA', 'outOfVocab': 'IN_BUILTIN_DICT', 'probability': 0.0}, {'text': {'content': '시', 'beginOffset': 3, 'length': 1}, 'tag': 'EP', 'probability': 0.99986386, 'outOfVocab': 'IN_WORD_EMBEDDING'}, {'text': {'content': '어요', 'beginOffset': 3, 'length': 2}, 'tag': 'EF', 'probability': 0.9903047, 'outOfVocab': 'IN_WORD_EMBEDDING'}, {'text': {'content': '.', 'beginOffset': 5, 'length': 1}, 'tag': 'SF', 'probability': 0.99277085, 'outOfVocab': 'IN_WORD_EMBEDDING'}], 'lemma': '안녕하', 'tagged': '안녕하/VA+시/EP+어요/EF+./SF', 'modified': ''}, {'text': {'content': '정말', 'beginOffset': 7, 'length': 2}, 'morphemes': [{'text': {'content': '정말', 'beginOffset': 7, 'length': 2}, 'tag': 'MAG', 'probability': 0.9925209, 'outOfVocab': 'IN_WORD_EMBEDDING'}], 'lemma': '정말'

In [13]:
# get tuple of pos tagging.
pa = res.pos()
print(pa)
# another methods
ma = res.morphs()
print(ma)
na = res.nouns()
print(na)
va = res.verbs()
print(va)

[('안녕하', 'VA'), ('시', 'EP'), ('어요', 'EF'), ('.', 'SF'), ('정말', 'MAG'), ('좋', 'VA'), ('은', 'ETM'), ('날씨', 'NNG'), ('이', 'VCP'), ('네', 'EF'), ('요', 'JX'), ('.', 'SF')]
['안녕하', '시', '어요', '.', '정말', '좋', '은', '날씨', '이', '네', '요', '.']
['날씨']
[]


###### mecab 설치

아래는 curl 명령어를 사용하여 Github에서 다운받아 mecab을 설치하는 과정입니다.

그리고 Mecab 라이브러리를 불러와 형태소 분석 기능을 테스트합니다.

In [ ]:
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [15]:
from konlpy.tag import Mecab

mecab = Mecab()

In [16]:
mecab.pos("안녕하세요")

[('안녕', 'NNG'), ('하', 'XSV'), ('세요', 'EP+EF')]

### 형태소 분석 테스트

형태소 분석기 바른과 mecab의 분석 성능을 테스트하는 간단한 코드입니다.

분석 성능 비교를 위해 바른에서 구축한 모호성 평가 데이터세트(https://github.com/bareun-nlp/korean-ambiguity-data) 를 다운받고 불러옵니다.

##### bareun

In [17]:
def dataload(path):
  """
  json파일을 불러와서 정답, 문장으로 전처리합니다.
  :param path: 평가 데이터의 path
  :return: 평가 데이터세트
  """
  import json
  with open(path) as f:
      sents = json.load(f)
  ret = []
  for idx, sent in enumerate(sents['sentences']):
      final_answer = []
      if '+' in sent['answer']:
          tokens = sent['answer'].split('+')
          temp = []
          for i in range(len(tokens)):
              form = tokens[i].split('/')[0]
              tag = tokens[i].split('/')[1]
              t = (form, tag)
              temp.append(t)
          final_answer.append(temp)
          final_answer.append(sent['text'])
      else:
          tokens = sent['answer'].split('/')
          form, tag = tokens[0], tokens[1]
          temp = (form, tag)
          final_answer.append(temp)
          final_answer.append(sent['text'])
      ret.append(final_answer)
  return ret


In [ ]:
!pip install -U gdown

In [ ]:
!gdown  https://drive.google.com/file/d/1TwJB5Bp12GJh54ip2cUFzfzOA7fluMN1/view?usp=share_link -O ambiguity_sentences.json --fuzzy

In [20]:
testset = dataload("./ambiguity_sentences.json")

In [21]:
print(testset[:5])

[[[('쿠키', 'NNG'), ('와', 'JC')], '밀가루, 계란, 우유 반죽과 한참을 씨름한 끝에 마침내 오븐에서 결혼 전 발레리나였던 아내의 모습을 본 뜬 쿠키와 곧 태어날 아기에게 먹일 젖병 모양의 쿠키가 나왔다.'], [[('쿠키', 'NNG'), ('와', 'JKB')], '뭐 케이크 빵 쿠키와 같은 베이커립니다.'], [[('퀄리티', 'NNG'), ('가', 'JKS')], '고려은단은 고가의 영국산 원료를 사용함에도 불구하고 원료의 대량 구매 및 최신 자동화 설비를 통한 원가 절감으로 가격 대비 퀄리티가 높다는 평가를 받고 있다.'], [[('퀄리티', 'NNG'), ('가', 'JKC')], '이 정도는 쫌 어느 정도 퀄리티가 되는 글이 나와야 책으로 낼 수 있다.'], [('퀸', 'NNG'), '대상 및 특별상 수상자는 12월 여성가족부가 주최하는 ‘아이디어 퀸 공모전’ 본선에 참가하게 된다.']]


바른 모델을 평가하는 함수입니다.

In [22]:
def bareun_evaluate(dataset):
  from tqdm import tqdm
  from google.protobuf.json_format import MessageToDict
  acc, tot = 0, 0
    
  for index, (answer, exam) in enumerate(tqdm(dataset)):
      correct = False
      result = []
      analyzed = MessageToDict(t.tags([exam]).msg())
      for word in analyzed['sentences'][0]['tokens']:
          for morph in word['morphemes']:
              result.append((morph['text']['content'], morph['tag']))
      answer_length = len(answer)
      correct_cnt = 0
      for idx, token in enumerate(result):
          # 첫번째 토큰이 정답이면 정답의 길이만큼 다음 토큰을 계산한다.
          i = 0
          if isinstance(answer, list): # 정답이 두개 이상의 토큰이 경우(ex. [(쿠키, NNG), (와, JC)])
              if (token[0] == answer[i][0]) and (token[1] == answer[i][1]):
                  correct_cnt += 1
                  while i <= answer_length:
                      i += 1
                      if i < answer_length:
                          try:
                              if (result[idx+i][0] == answer[i][0]) and (result[idx+i][1] == answer[i][1]):
                                  correct_cnt +=1
                                  if correct_cnt == answer_length:
                                      correct = True
                                      break
                          except: # 정답이 결과보다 많은 경우는 분석을 잘못한 경우이니 틀린 것이다.
                              correct = False
                              break
          else: # 정답이 하나의 토큰인 경우(ex.밀가루, NNG)
              if (token[0] == answer[0]) and (token[1] == answer[1]):
                  correct = True

      acc += int(correct)
      tot += 1
      
  return acc / tot

In [23]:
bareun_probability = bareun_evaluate(testset)

100%|██████████| 35396/35396 [09:03<00:00, 65.11it/s]


In [ ]:
print(f'정확도: {bareun_probability*100:.2f}%')

##### mecab

mecab 모델을 평가하는 함수입니다.

In [25]:
def mecab_evaluate(dataset):
  from tqdm import tqdm
  from google.protobuf.json_format import MessageToDict
  acc, tot = 0, 0
    
  for index, (answer, exam) in enumerate(tqdm(dataset)):
      correct = False
      result = mecab.pos(exam)
      answer_length = len(answer)
      correct_cnt = 0
      for idx, token in enumerate(result):
          # 첫번째 토큰이 정답이면 정답의 길이만큼 다음 토큰을 계산한다.
          i = 0
          if isinstance(answer, list): # 정답이 두개 이상의 토큰이 경우(ex. [(쿠키, NNG), (와, JC)])
              if (token[0] == answer[i][0]) and (token[1] == answer[i][1]):
                  correct_cnt += 1
                  while i <= answer_length:
                      i += 1
                      if i < answer_length:
                          try:
                              if (result[idx+i][0] == answer[i][0]) and (result[idx+i][1] == answer[i][1]):
                                  correct_cnt +=1
                                  if correct_cnt == answer_length:
                                      correct = True
                                      break
                          except: # 정답이 결과보다 많은 경우는 분석을 잘못한 경우이니 틀린 것이다.
                              correct = False
                              break
          else: # 정답이 하나의 토큰인 경우(ex.밀가루, NNG)
              if (token[0] == answer[0]) and (token[1] == answer[1]):
                  correct = True

      acc += int(correct)
      tot += 1
      
  return acc / tot

In [26]:
mecab_probability = mecab_evaluate(testset)

100%|██████████| 35396/35396 [00:06<00:00, 5772.57it/s]


In [ ]:
print(f'정확도: {mecab_probability*100:.2f}%')

##### 비교

In [28]:
sents = [
"조길상의 음성은 김광석보다 조금 탁하고 가는 편이다.",
"그건 제가 그냥 갖고 있는 개인 킵니다.",
"이런 속도라면 20년 뒤에는 50만에서 200만 종이 멸종될 것으로 학자들은 예측하고 있다.",
"방안 가득 풀어놓으며 먹자고 했지만 화가 날 대로 나 있던 나는 ",
"어제 귀가 시간부터 오늘 아침까지는 남편으로, 아들로, 아빠로 지냈지만",
"회사가 진심이라면 먼저 성의 있는 안을 내놔야 한다”고 말했다.",
"한반도를 둘러싼 주변 네 강의 변화는 격세지감이 있다.",
"창의 융합 콘텐츠 개발을 위해",
"두 번째는 모두를 위한 창의 인성 교육을 확산시키겠다 하는거고요"
]
bareun_result = []
mecab_result = []
for sent in sents:
  res1 = t.tags([sent]).pos()
  bareun_result.append(res1)
  res2 = mecab.pos(sent)
  mecab_result.append(res2)


| 표면형 | 정답          | bareun 분석결과      | mecab 분석결과      |
|-----|-------------|-------------|-------------|
| 가는  | 가늘/VV       | 가늘/VV       | 가/VV        |
| 키   | 키/NNG       | 키/NNG       | 키/NNP       |
| 종이  | 종/NNG+이/JKS       | 종/NNG+이/JKS       | 종이/NNG      |
| 귀가  | 귀가/NNG      | 귀가/NNG      | 귀/NNG+가/JKS |
| 성의  | 성의/NNG      | 성의/NNG      | 성/NNG+의/JKG |
| 강의  | 강/NNG+의/JKG | 강/NNG+의/JKG | 강의/NNG      |
| 창의  | 창의/NNG      | 창의/NNG      | 창/NNG+의/JKG |
| 창의  | 창의/NNG      | 창의/NNG      | 창/NNG+의/JKG |

In [29]:
for result in bareun_result:
  print(result)

[('조길상', 'NNP'), ('의', 'JKG'), ('음성', 'NNG'), ('은', 'JX'), ('김광석', 'NNP'), ('보다', 'JKB'), ('조금', 'MAG'), ('탁하', 'VA'), ('고', 'EC'), ('가늘', 'VA'), ('ㄴ', 'ETM'), ('편', 'NNB'), ('이', 'VCP'), ('다', 'EF'), ('.', 'SF')]
[('그거', 'NP'), ('ㄴ', 'JX'), ('제', 'NP'), ('가', 'JKS'), ('그냥', 'MAG'), ('갖', 'VV'), ('고', 'EC'), ('있', 'VX'), ('는', 'ETM'), ('개인', 'NNG'), ('키', 'NNG'), ('이', 'VCP'), ('ㅂ니다', 'EF'), ('.', 'SF')]
[('이런', 'MMD'), ('속도', 'NNG'), ('이', 'VCP'), ('라면', 'EC'), ('20', 'SN'), ('년', 'NNB'), ('뒤', 'NNG'), ('에', 'JKB'), ('는', 'JX'), ('50', 'SN'), ('만', 'NR'), ('에서', 'JKB'), ('200', 'SN'), ('만', 'NR'), ('종', 'NNG'), ('이', 'JKS'), ('멸종되', 'VV'), ('ㄹ', 'ETM'), ('것', 'NNB'), ('으로', 'JKB'), ('학자', 'NNG'), ('들', 'XSN'), ('은', 'JX'), ('예측하', 'VV'), ('고', 'EC'), ('있', 'VX'), ('다', 'EF'), ('.', 'SF')]
[('방', 'NNG'), ('안', 'NNG'), ('가득', 'MAG'), ('풀어놓', 'VV'), ('으며', 'EC'), ('먹', 'VV'), ('자고', 'EC'), ('하', 'VV'), ('았', 'EP'), ('지만', 'EC'), ('화', 'NNG'), ('가', 'JKS'), ('나', 'VV'), ('ㄹ', 'ETM'), ('대로

In [30]:
for result in mecab_result:
  print(result)

[('조길상', 'NNP'), ('의', 'JKG'), ('음성', 'NNG'), ('은', 'JX'), ('김광석', 'NNP'), ('보다', 'JKB'), ('조금', 'MAG'), ('탁하', 'VV'), ('고', 'EC'), ('가', 'VX'), ('는', 'ETM'), ('편', 'NNB'), ('이', 'VCP'), ('다', 'EF'), ('.', 'SF')]
[('그건', 'NP+JX'), ('제', 'NP'), ('가', 'JKS'), ('그냥', 'MAG'), ('갖', 'VV'), ('고', 'EC'), ('있', 'VX'), ('는', 'ETM'), ('개인', 'NNG'), ('킵', 'NNP'), ('니다', 'VCP+EF'), ('.', 'SF')]
[('이런', 'MM'), ('속도', 'NNG'), ('라면', 'VCP+EC'), ('20', 'SN'), ('년', 'NNBC'), ('뒤', 'NNG'), ('에', 'JKB'), ('는', 'JX'), ('50', 'SN'), ('만', 'NNG'), ('에서', 'JKB'), ('200', 'SN'), ('만', 'NR'), ('종이', 'NNG'), ('멸종', 'NNG'), ('될', 'XSV+ETM'), ('것', 'NNB'), ('으로', 'JKB'), ('학자', 'NNG'), ('들', 'XSN'), ('은', 'JX'), ('예측', 'NNG'), ('하', 'XSV'), ('고', 'EC'), ('있', 'VX'), ('다', 'EF'), ('.', 'SF')]
[('방안', 'NNG'), ('가득', 'MAG'), ('풀어놓', 'VV'), ('으며', 'EC'), ('먹', 'VV'), ('자고', 'EC'), ('했', 'VX+EP'), ('지만', 'EC'), ('화', 'NNG'), ('가', 'JKS'), ('날', 'VV+ETM'), ('대로', 'NNB'), ('나', 'VV+EC'), ('있', 'VX'), ('던', 'ETM'), ('나',